In [1]:
#https://keras.io/examples/generative/vae/
#https://keras.io/examples/generative/molecule_generation/

In [2]:
# !python3 -m pip install --upgrade pip
# !pip install pydot
# !apt-get install -y graphviz

In [3]:
import os,shutil,random
from sklearn.preprocessing import MinMaxScaler
import joblib
from IPython.display import clear_output
import pandas as pd
import h5py
import tensorflow as tf
from IPython.display import clear_output,display, HTML
from sklearn.model_selection import train_test_split
import datetime as dt
import numpy as np
from src.model import Sampling
#================== initialization ==================
currentTM=dt.datetime.now().strftime("%Y-%m-%dT%H%M%S")
PROJECT = "testVAEModel"
LATENT_DIM = 32
VAE_LR = 0.001
EPOCHS = 10
BATCH_SIZE = 32

PARQUET_PATH = './data/OptionsEOD_STG.parquet'
SCALER_PATH = './data/scaler/scaler.gz'
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']
SCALER_COL  = ['DTE','INTRINSIC_VALUE', 'TOTAL_VOLUME',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME' ]
MODEL_PATH = "./models/"
H5_PATH = './data/OptTrainData/'
DISPLAY = False
WANDB_LOG = True
RESUME = False

Scaler = joblib.load(SCALER_PATH )

2024-07-11 15:05:52.289428: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
notes = f"""
test Run use_bias set false , no tranfrom
"""
import wandb
from wandb.keras import WandbCallback
CONFIG = {"latent_dim":LATENT_DIM,
          "learning_rate": VAE_LR,
          "epochs": EPOCHS,
          "batch_size": BATCH_SIZE,
          "architecture": "VAE",
          "dataset": "OptionsChaine",
          "encoder_dense_units":[128,64],
          "encoder_dropout_rate":0.2,
          "decoder_dense_units":[64, 128],
          "decoder_dropout_rate":0.2,
          "use_bias":False,
          "transform":True
           }

if WANDB_LOG :
    wandb.login()
    run = wandb.init(project=PROJECT, 
                     name=currentTM, 
                     config=CONFIG,
                     notes=notes
                    )

wandb: Currently logged in as: wasan-sinlapa. Use `wandb login --relogin` to force relogin


In [5]:
#Example

# from IPython.display import clear_output,display, HTML
# import numpy as np
# #load scaler
# scaler = MinMaxScaler()
# PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
# random.shuffle(PartitionDate)
# scaler = joblib.load(SCALER_PATH)


# for i,partdate in enumerate(PartitionDate) :
#     df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
#                                  , filters=[('PartitionDate', '=', partdate)]
#                                 )
#     df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
#     df['C_VOLUME'] = df['C_VOLUME'].fillna(0)
#     DATA  = np.empty((0,) + (20,9) ) 
#     for opt_id in np.unique( df[["OPTIONS_ID"]].values):
#         df_filter  = df[df["OPTIONS_ID"]==opt_id]
#         if len(df_filter) == 20:
#             DATA = np.vstack((DATA ,[scaler.transform(df_filter[SCALER_COL])]))
#         else:
#             #print( len(df_filter) )
#             #display(HTML(df_filter[['STRIKE']+SCALER_COL].to_html()))
#             pass
            
#     ## Save the NumPy array to an HDF5 file
#     # with h5py.File(H5_PATH+f"{partdate}.h5", 'w') as f:
#     #     dset = f.create_dataset(f'{partdate}', data=DATA, chunks=True , compression='gzip')

#     print(f"[Processing] {partdate}, {round(((i+1)/len(PartitionDate))*100,2)}%     ",end='\r')

In [6]:
# # Save the NumPy array to an HDF5 file
# with h5py.File(H5_PATH, 'w') as f:
#     #dset = f.create_dataset('dataset', data=DATA, chunks=True, compression='gzip')
#     #test
#     dset = f.create_dataset('dataset', data=DATA, compression='gzip')

In [7]:
#=====================================================================

In [8]:
#SCALER_COL  = ['DTE','INTRINSIC_VALUE', 'TOTAL_VOLUME',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME' ]
select_x = [i for i,c in  enumerate(SCALER_COL) if c in ['DTE','INTRINSIC_VALUE'] ]
select_y = [i for i,c in enumerate(SCALER_COL) if c in ['C_BID',	'C_ASK',  'P_BID',	'P_ASK'] ]

In [9]:
from src.model import OptionChainGenerator
from src.layer import encoder, decoder

model = OptionChainGenerator(
    encoder(latent_dim = LATENT_DIM, 
            input_shape= (32,len(select_x) ), 
            dense_units = CONFIG["encoder_dense_units"], 
            dropout_rate= CONFIG["encoder_dropout_rate"],
            use_bias=CONFIG["use_bias"]
           ), 
    decoder(latent_dim  = LATENT_DIM , 
            output_shape= (32,len(select_y) ),
            dense_units = CONFIG["decoder_dense_units"],
            dropout_rate= CONFIG["decoder_dropout_rate"],
            use_bias=CONFIG["use_bias"]
           )
)

def dummy_loss(y_true, y_pred):
    return 0.0
    
vae_optimizer = tf.keras.optimizers.Adam(learning_rate=VAE_LR)
model.compile(vae_optimizer )#, loss=dummy_loss)

In [10]:
################## show model ######################
if DISPLAY :
    from tensorflow.keras.utils import model_to_dot
    from IPython.display import SVG, display
    
    def display_model(model, width=1024, height=512):
        dot = model_to_dot(model, show_shapes=True, show_layer_names=True)
        svg_data = dot.create(prog='dot', format='svg').decode("utf-8")
        svg_html = f'<div style="width:{width}px;height:{height}px;">{svg_data}</div>'
        display(HTML(svg_html))

In [11]:
## Example usage:
## Display the encoder model with reduced size
if DISPLAY :
    display_model(model.encoder, width=1024, height=512)

In [12]:
if DISPLAY :
    display_model(model.decoder, width=2500, height=512)

In [13]:
#================== loadmodel ====================

In [14]:
from tensorflow.keras.models import load_model
model_path = MODEL_PATH+f'{PROJECT}'
if not RESUME :
    if os.path.exists(model_path) :
        shutil.rmtree(model_path)
if not os.path.exists(model_path):
    os.makedirs(model_path)
    model.encoder.save(model_path+f'/'+f'encoder.keras') 
    model.decoder.save(model_path+f'/'+f'decoder.keras') 
else:
    model.encoder = load_model(model_path+'/'+f'encoder.keras') 
    model.decoder = load_model(model_path+'/'+f'decoder.keras') 

In [15]:
#================== train model ==================
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)

STOP_MODEL = False
STACK_DATA = np.empty((0,) + (32,9) ) #init STACK_DATA

for partdate in PartitionDate[:] :
    clear_output(wait=False)
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        DATA = f[partdate][:]
    data_shape = DATA.shape
    ###transform
    if CONFIG['transform'] :
        DATA = Scaler.transform(DATA.reshape(-1,data_shape[-1]))
        DATA = DATA.reshape(data_shape)
    DATA = np.vstack((DATA ,STACK_DATA))
    if len(DATA) < 64 :
        #stack data
        STACK_DATA = np.vstack((STACK_DATA ,DATA))
    else: 
        # if DATA.isna().sum().sum() > 0:
        #     print("Data contains NaNs. Please handle them before scaling.")
        STACK_DATA = np.empty((0,) + (32,9) )
        X = DATA[:, :, select_x]  # เลือกข้อมูลแถวแรกถึงแถวที่ 3 สำหรับ X
        Y = DATA[:, :, select_y]  # เลือกข้อมูลแถวที่ 3 เป็นต้นไปสำหรับ Y
        x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
        random.shuffle(PartitionDate)
        tf.keras.backend.clear_session() 
        history = model.fit(x_train , y_train, epochs=CONFIG['epochs'], batch_size=BATCH_SIZE, validation_data=(x_val, y_val) )
        #history = model.fit(x_train , y_train, epochs=5, batch_size=BATCH_SIZE )
        if  np.isnan(  np.average( history.history['kl_loss'] )  ) or np.isnan(  np.average( history.history['val_kl_loss'] )  ):
            STOP_MODEL = True 
            print(x_train)
            print("---")
            print(x_val)
            print("=============")
        if WANDB_LOG :
            LogKeys = history.history.keys()
            LogVal={}
            for k in LogKeys:  
                LogVal[k] = np.average(  history.history[k] )
            wandb.log(LogVal, commit=True)
        
    if STOP_MODEL :
        break
    
            
    model.encoder.save(model_path+f'/'+f'encoder.keras') 
    model.decoder.save(model_path+f'/'+f'decoder.keras') 
if WANDB_LOG : wandb.finish()

Epoch 1/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - kl_loss: 3.7786e-06 - total_loss: 1.7322 - val_kl_loss: 2.4195e-08 - val_total_loss: 1.5279
Epoch 2/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - kl_loss: 1.5650e-05 - total_loss: 1.7991 - val_kl_loss: 8.7558e-05 - val_total_loss: 1.5253
Epoch 3/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - kl_loss: 1.2766e-04 - total_loss: 1.7670 - val_kl_loss: 8.4603e-05 - val_total_loss: 1.5191
Epoch 4/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - kl_loss: 1.1371e-04 - total_loss: 1.6534 - val_kl_loss: 1.5133e-04 - val_total_loss: 1.5235
Epoch 5/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - kl_loss: 3.1321e-04 - total_loss: 1.5607 - val_kl_loss: 1.8881e-04 - val_total_loss: 1.5221
Epoch 6/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - kl_loss: 2.1626e-04 - total_loss: 1.6680 - val_kl_loss: 2.0927e-04 - val_total_loss: 1.5130
Epoch 7/10



KeyboardInterrupt



In [ ]:
`====================================================

In [ ]:
======================== predict =========================

In [1]:
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)
for partdate in PartitionDate[:1] :
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        DATA = f[partdate][:]
    data_shape = DATA.shape
    if CONFIG['transform'] :
        DATA_TF = Scaler.transform(DATA.reshape(-1,data_shape[-1]))
        DATA_TF = DATA_TF.reshape(data_shape)

NameError: name 'os' is not defined

In [ ]:
DATA_TF.shape

In [ ]:
X = DATA_TF[:, :, select_x][:1]
Y_real = DATA_TF[:, :, select_y][:1]
# X data
df = pd.DataFrame(
    DATA[:, :, :3][:1].reshape(32, 3), 
    columns=SCALER_COL[:3])
#print(df)

In [2]:
z_mean, log_var = model.encoder(X) 
z = Sampling()([z_mean, log_var])
decode_data = model.decoder(z)

NameError: name 'model' is not defined

In [26]:
decode_data

<tf.Tensor: shape=(1, 32, 4), dtype=float32, numpy=
array([[[0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.7234566 , 0.72337925, 0.7233077 , 0.7233854 ],
        [0.689044  , 0.68909174, 0.6890693 , 0.6890581 ],
        [0.6689366 , 0.66889364, 0.668935  , 0.6688881 ],
        [0.78139985, 0.781265  , 0.7814391 , 0.7812704 ],
        [0.7381041 , 0.73812735, 0.7379848 , 0.7379633 ],
        [0.69196725, 0.69200623, 0.6919941 , 0.69195676],
        [0.6634506 , 0.6632937 , 0.66315794, 0.66328907],
        [0.6242391 , 0.6240769 , 0.6243938 , 0.62421733],
        [0.5877602 , 0.5886376 , 0.5882968 , 0.58698463],
        [0.5496805 , 0.5484245 , 0.55076563, 0.54961896],
        [0.5212235 , 0.        , 0.5222677 , 0.5223665 ],
        [0.51650155,

In [30]:
Y_real

array([[[-0.434937  , -0.43490119, -0.3737459 , -0.24608101],
        [-0.434937  , -0.43490119, -0.3737459 , -0.24608101],
        [-0.434937  , -0.43490119, -0.3737459 , -0.24608101],
        [-0.434937  , -0.43490119, -0.3737459 , -0.24608101],
        [-0.434937  , -0.43490119, -0.3737459 , -0.24608101],
        [-0.40191274, -0.40153765, -0.3737459 , -0.24608101],
        [-0.40470174, -0.404874  , -0.3737459 , -0.24594222],
        [-0.40825137, -0.40790143, -0.3737459 , -0.24594222],
        [-0.41154746, -0.41074351, -0.37352711, -0.24594222],
        [-0.41389275, -0.41401808, -0.3736365 , -0.24587282],
        [-0.41699868, -0.4171073 , -0.37352711, -0.24594222],
        [-0.42035815, -0.42013473, -0.37341772, -0.24587282],
        [-0.42308376, -0.42322395, -0.37308954, -0.24559523],
        [-0.42599953, -0.42594246, -0.37276136, -0.24531765],
        [-0.42859837, -0.42859919, -0.37166743, -0.24469308],
        [-0.43081689, -0.43076164, -0.37013594, -0.24365213],
        

In [ ]:
======================= _compute_loss ==============================

In [ ]:
#generated_data = [c_bid, c_ask, c_volume, p_bid, p_ask, p_volume]
colList = ["c_bid", "c_ask", "c_volume", "p_bid", "p_ask", "p_volume"]
generated_data = decode_data[3:]
z_mean    = z_mean
z_log_var = log_var
Y_real    = DATA[:, :, 3:][:1]

In [ ]:
for  col,genData in zip(colList,generated_data):
    print( colList.index(col),col )

In [ ]:
subtract_genData = genData - tf.cast(tf.expand_dims(Y_real[:, :, colList.index(col)], axis=-1)
        , tf.float32) 

In [ ]:
reconstruction_values_total = []
reconstruction_values_total.append( tf.reduce_mean( tf.square(subtract_genData)   ) )

In [ ]:
log_var = tf.clip_by_value(log_var, -1.0, 1.0)
kl_loss = -0.5 * tf.reduce_sum(1 + log_var - tf.square(z_mean) - tf.exp(log_var), axis=-1)

In [ ]:
tf.reduce_mean(reconstruction_values_total + kl_loss)

In [ ]:
========== kiras vae origi

In [ ]:
Y_real[0][0]

In [ ]:
 tf.concat(decode_data, axis=-1).numpy()[0][0]

In [ ]:
Y_real[0] -  tf.concat(decode_data, axis=-1).numpy()

In [ ]:
features_loss = tf.reduce_mean(
    tf.reduce_sum(
        tf.keras.losses.mean_squared_error(Y_real, tf.concat(decode_data, axis=-1)),
        axis=(1),
    )
)

In [ ]:
features_loss

In [ ]:
# features_loss = tf.reduce_mean(
#     tf.reduce_sum(
#         keras.losses.categorical_crossentropy(features_real, features_gen),
#         axis=(1),
#     )
# )
# kl_loss = -0.5 * tf.reduce_sum(
#     1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), 1
# )
# kl_loss = tf.reduce_mean(kl_loss)

# property_loss = tf.reduce_mean(
#     keras.losses.binary_crossentropy(qed_true, qed_pred)
# )

# graph_loss = self._gradient_penalty(graph_real, graph_generated)

# return kl_loss + property_loss + graph_loss + adjacency_loss + features_loss

In [ ]:
======================= inverse_transform ========================

In [ ]:
#add 0
decode_data = [tf.zeros([1, 32, 1])]*3 + decode_data

In [ ]:
invert_decode = Scaler.inverse_transform(
    np.array([d.numpy().reshape(-1) for d in decode_data]).transpose()
    ) 

In [ ]:
pd.DataFrame(
    invert_decode[:,3:], 
    columns=SCALER_COL[3:])

In [ ]:
=====================================================================